In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torch.nn import functional as F
# from efficientnet_pytorch.model import EfficientNet
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm_notebook
from PIL import Image
device = 'cuda'

In [2]:
test_data_path = '../data/test_masked/*'
test_images = glob(test_data_path)

In [3]:
test_images_lst = [i.split('/')[-1] for i in test_images]

In [4]:
class TestDataLoader:
    def __init__(self, image_paths, preprocess):
        self.image_paths = image_paths
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image_path = self.image_paths[item]
        img = Image.open(image_path)
        
        if preprocess:
            img = preprocess(img)
        
        return {
            'image' : img
        }

In [5]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
preprocess = transforms.Compose(
[
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [6]:
test_dataset = TestDataLoader(test_images, preprocess)

In [7]:
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

In [8]:
def predict(data_loader, model, device='cuda'):
    model.eval()
    final_predictions = np.array([])
    with torch.no_grad():
        tk0 = tqdm_notebook(data_loader, total=len(data_loader))
        for b_idx, data in enumerate(tk0):
            for key, value in data.items():
                data[key] = value.to(device)
            predictions = model(**data)
            predictions = predictions.cpu().numpy()
#             print(predictions)
            final_predictions = np.append(final_predictions, predictions)
    return torch.tensor(final_predictions)

In [9]:
# Resnet
import pretrainedmodels
class ResNet18(nn.Module):
    def __init__(self, pretrained="imagenet"):
        super(ResNet18, self).__init__()
        self.model = pretrainedmodels.__dict__[
            "resnet18"
        ](pretrained=pretrained)
        self.out = nn.Linear(512, 1)

    def forward(self, image):
        bs, _, _, _ = image.shape
        x = self.model.features(image)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.reshape(bs, -1)
        out = self.out(x)
        return out

model = ResNet18()
model.load_state_dict(torch.load('wandb/run-20200719_041321-1e842ylc/model.tar')['model_state_dict'])
# model.load_state_dict(torch.load('/home/ashwin/wd/padhai/model(2).tar')['model_state_dict'])

<All keys matched successfully>

In [11]:
# Resnet
import pretrainedmodels
class ResNet50(nn.Module):
    def __init__(self, pretrained="imagenet"):
        super(ResNet50, self).__init__()
        self.model = pretrainedmodels.__dict__[
            "resnet50"
        ](pretrained=pretrained)
        
        self.out = nn.Sequential(
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.25),
            nn.Linear(2048, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.5),
            nn.Linear(512, 1)
        )

    def forward(self, image):
        bs, _, _, _ = image.shape
        x = self.model.features(image)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.reshape(bs, -1)
        out = self.out(x)
        return out

model = ResNet50()
model.load_state_dict(torch.load('wandb/run-20200718_070618-1a0xkz77/model.tar')['model_state_dict'])
# model.load_state_dict(torch.load('/home/ashwin/wd/padhai//model(3).tar')['model_state_dict'])

<All keys matched successfully>

In [10]:
model.to(device)
pred0 = predict(test_loader, model)

<ipython-input-8-478c83c16726>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tk0 = tqdm_notebook(data_loader, total=len(data_loader))


In [11]:
final_pred = pred0
final_pred

tensor([216.5544, 199.7953, 340.1466,  ..., 123.5930, 342.0819, 339.0950],
       dtype=torch.float64)

In [38]:
final_pred = (pred0 + pred1 + pred2+ pred3 + pred4)/5

In [12]:
submission = pd.DataFrame({'filename':test_images_lst,'xRot': final_pred})
submission.head()

,filename,xRot
0,009999.png,216.554382
1,010000.png,199.795273
2,010001.png,340.146606
3,010002.png,125.332909
4,010003.png,147.055099


In [13]:
submission.to_csv('resnet18_onefold.csv', index=False)